# Intel MPI Benchmarks

Results of selected MPI1 tests from the [IMB benchmark suite](https://software.intel.com/content/www/us/en/develop/documentation/imb-user-guide/top.html).

All tests use two nodes:
- pingpong: Single process per node
- uniband: Multiple processes per node, communicating in pairs split across nodes
- biband: As for uniband but communication initiated in both directions
- alltoall: As many processes on each node as there are (hardware) cores

Tests use a range of message sizes and report bandwidth and/or latency.

In [ ]:
# Plotting setup
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import ticker
from IPython.display import display, HTML

import pandas as pd
import os, sys
from pprint import pprint
sys.path.extend(('../../reframe', '../../')) # the `modules` package imports `reframe` so need that in sys.modules
import modules

# Bandwidth/latency vs message size

The plots below show results vs message size for selected tests:
- pingpong
- 2x process runs of uniband and biband
- alltoall (using all cores on each node)

Results are for the most-recent test on each system/partition/environment combination.

In [ ]:
for (test, column, param) in [('IMB_PingPong', 'Mbytes/sec', 'bandwidth'),
                              ('IMB_PingPong', 't[usec]', 'latency'),
                              ('IMB_Biband_2', 'Mbytes/sec', 'bandwidth'),
                              ('IMB_Uniband_2', 'Mbytes/sec', 'bandwidth'),
                             ]:
    
    # find all data and work out differences between them:
    paths = modules.utils.find_run_outputs(root='../../output', test=test)
    fig, ax = plt.subplots(nrows=1, ncols=1)
    meta = [modules.utils.parse_path_metadata(p) for p in paths]
    
    # load and plot:
    for ix, path in enumerate(paths):
        datas = modules.imb.read_imb_out(path)
        assert len(datas) == 1
        data = datas[list(datas.keys())[0]]
        label = '%s:%s' % (meta[ix]['sysname'], meta[ix]['partition'])
        plt.plot(data['#bytes'], data[column], 'o-', data=data, label=label)
        
    # format plot:
    ax.set_xscale('log', basex=2)
    if param == 'latency':
        ax.set_yscale('log', basey=10)
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: modules.utils.sizeof_fmt(x)))
    ax.grid(True, which="both")
    ax.set_xlabel('#bytes')
    ax.set_ylabel(column)
    ax.set_title('%s: %s' % (test, param))
    ax.legend()

# Max Bandwidth vs Number of Processes per Node

The plots below show the **maximum** bandwidth for tests using varying numbers of processes per node. Results are for the most-recent test on each system/partition/environment combination.

In [ ]:
# constants:
Gb_to_MB = 125.0
TEST_BANDWIDTH_FACTOR = {'IMB_Uniband_*':1.0, 'IMB_Biband_*':2.0} # b/c biband is bidirectional

for test in ['IMB_Uniband_*', 'IMB_Biband_*']:
    
    # get, plot and tabulate data:
    df = modules.utils.tabulate_last_perf(test, 'procs_per_node', 'max_bandwidth')
    print('%s: Max bandwidth (MB)' % test)
    display(HTML(df.to_html()))
    ax = df.plot(grid=True, style='o-')
    
    # add max bandwidth info to plot:
    xlim = ax.get_xlim()
    for syspart, bw in modules.utils.get_sys_param('interconnect_Gb').items():
        theoretical_bw_MB = bw * Gb_to_MB * TEST_BANDWIDTH_FACTOR[test]
        label = syspart + " (theoretical b/w)"
        ax.plot(xlim, [theoretical_bw_MB, theoretical_bw_MB], '--', label=label)
    
    # format plot:
    ax.set_xlabel('processes per node')
    ax.set_ylabel('max bandwidth (MB)')
    plt.tight_layout()
    ax.set_title(test)
    ax.legend()
    
    # TODO: calculate real vs theortical perf
    print('%s: Max bandwidth as %% of theoretical bandwidth' % test)
    theory = [modules.utils.get_sysinfo(syspart)['interconnect_Gb'] * Gb_to_MB * TEST_BANDWIDTH_FACTOR[test] for syspart in df.columns]
    df_percent = 100 * (df / theory)
    #print('theory:', theory)
    display(HTML(df_percent.astype(int).to_html()))
    
    print('-----------------------------------------------')
